### コサイン類似度とは
TF-IDF値同士のベクトル間距離を測ることで文書の近さを計算する。  
そのベクトル間距離を計測する方法  
三角関数のコサインを利用する  
- コサイン類似度が1に近いほど「ベクトル同士の成す角度が小さい」＝類似している
- コサイン類似度が0に近いほど「ベクトル同士が直行する」＝類似していないとなる。  
  
TF-IDFの場合、共通する単語が多いほど、大きくなり、TF-IDF値の高い単語が共通しているほど大きくなる。

$$
\begin{eqnarray}
xとyのコサイン類似度 &=& \displaystyle \frac{ xとyの内積 }{ xの絶対値 \times yの絶対値 } \\
    &=& \displaystyle \frac{ \displaystyle \sum_{ i = 1 }^{  } xiyi }{ \sqrt{ \displaystyle \sum_{ i = 1 }^{  } xi^2 } \times \sqrt{ \displaystyle \sum_{ i = 1 }^{  } yi^2 } }
\end{eqnarray}
$$

## コサイン類似度を利用し類似文書を検索する練習

### sqlite3操作用関数の読み込み

In [1]:
import json
import sqlite3

conn = None

# データベース接続
def connect():
    # global変数でconnを呼び出し
    global conn
    # データベースの場所を指定
    conn = sqlite3.connect('/vagrant/NaturalLanguageProcessing/data/sqlite3/sqlite3')

# データベース接続終了
def close():
    # 終了
    conn.close()

# テーブル作成
def create_table():
    # executeでSQL構文作成、docsがあれば削除
    conn.execute('DROP TABLE IF EXISTS docs')
    # docsテーブルを新規作成
    conn.execute('''CREATE TABLE docs (
            id          INTEGER PRIMARY KEY AUTOINCREMENT,
            content     TEXT,
            meta_info   BLOB,
            sentence    BLOB,
            chunk       BLOB,
            token       BLOB
        )''')

# データをインサートする
def load(values):
    # exucutemany()はvaluesに指定したパラメータ順序またはマッピングを?に入れて実行できる
    conn.executemany('INSERT INTO docs (content, meta_info) VALUES (?,?)', values)
    # 確定
    conn.commit()

# 一部のデータを見る
def get(doc_id, fl):
    #.fetchone()で指定した1行を取得
    row_ls = conn.execute(f"SELECT {','.join(fl)} FROM docs WHERE id = {doc_id}").fetchone()
    # row_ls = conn.execute('SELECT {} FROM docs WHERE id = ?'.format(','.join(fl)),(doc_id,)).fetchone()
    row_dict = {}
    # flとrow_lsで抜き出したデータをzipする
    for key, value in zip(fl, row_ls):
        row_dict[key] = value
    return row_dict

# id番号を抜き出す
def get_all_ids(limit, offset=0):
    return [record[0] for record in
            # limitで取得上限、OFFSETで開始位置を指定してデータを抜き出す。そのデータの1番目id番号を抜き出す
            conn.execute(f'SELECT id FROM docs LIMIT {limit} OFFSET {offset}')]
            # conn.execute('SELECT id FROM docs LIMIT ? OFFSET ?',(limit, offset))]

def set_annotation(doc_id, name, value):
    conn.execute(f'UPDATE docs SET {name} = {json.dumps(value)} where id = {doc_id}')
    conn.commit()

# アノテーションを取得
def get_annotation(doc_id, name):
    # docsのid行をwhere idで指定しnameから取り出す
    row = conn.execute(f'SELECT {name} FROM docs WHERE id = {doc_id}').fetchone()
    if row[0] is not None:
        return json.loads(row[0])
    else:
        return []

### TfidfVectorizerを使用し分析開始

In [4]:
# 単語の重要度を計算するTfidfVectorizerを読み込み
from sklearn.feature_extraction.text import TfidfVectorizer
# ベクトル間距離を利用したコサイン類似度の読み込み
from sklearn.metrics.pairwise import cosine_similarity

# データべース接続
connect()

data = []
doc_ids = []

# id番号をすべて抜き出す
for doc_id in get_all_ids(limit=-1):
    # 文書ごとに出現する単語の原型を取り出して、joinを使用し空白区切りでdata変数にappendで格納していく
    data.append(' '.join([token['lemma'] for token in get_annotation(doc_id, 'token')]))
    # 文書IDを保存しておく
    doc_ids.append(doc_id)


# TFIDFを計算するためにインスタンス化、analyzerは分析対象が単語なのかスペース区切りの文章なのかを指定する。max_dfは全文書中の何割以上の単語を無視するか？
vectorizer = TfidfVectorizer(analyzer='word', max_df=0.9)
# 引数で与えられた文書集合をもとに、各文書中の単語のTF-IDF値を計算する
vecs = vectorizer.fit_transform(data)

### コサイン類似度の計算

In [7]:
# vecsのコサイン類似度を計算する
sim = cosine_similarity(vecs)
'''
sklearnライブラリのcosine_similarity関数
入力された全文暑中の文書対すべての組み合わせに対し、類似度を計算する。
出力は行列の形
[2][3]の要素は、2番目の文書に対する、3番目の文書の類似度の事。
'''

# 0番目の文書に対して類似度の高い文書だけ表示する
docs = zip(doc_ids, sim[0])

# docsをコサイン類似度の高い順に並び替えて取り出し
for doc_id, similarity in sorted(docs, key=lambda x: x[1], reverse=True):
    # meta_infoをdoc_idから取り出し
    meta_info = json.loads(get(doc_id, ['meta_info'])['meta_info'])
    # meta_infoからtitleを取り出し
    title = meta_info['title']
    print(doc_id, title, similarity)
    
# データベース接続解除
close()

1 ボリビア 1.0
185 ペルー 0.3543020863309558
9 チリ 0.28952645763737483
102 パラグアイ 0.2303822698658134
168 アルゼンチン 0.21462497862892693
12 エクアドル 0.18377017377608187
135 スペイン 0.18224749193046635
36 コロンビア 0.17899204700796942
30 ブラジル 0.17729124009773817
26 ベネズエラ 0.15646912232949947
93 ウルグアイ 0.15366626178592194
64 アメリカ合衆国 0.13761214451919812
169 ホンジュラス 0.12216456850971538
55 日本 0.11927376278095436
86 ニカラグア 0.10966203918391114
45 コスタリカ 0.10388399507098711
124 ドミニカ共和国 0.10146683669933794
39 赤道ギニア 0.100448987737529
151 ポルトガル 0.09968439729660769
179 フランス 0.09795139977207272
196 キューバ 0.09585267073238923
153 トルクメニスタン 0.09548088886885911
184 エルサルバドル 0.09536571806890214
53 メキシコ 0.09384292793532331
174 ドイツ 0.09253578431825188
194 パナマ 0.0906205819324643
19 オーストラリア 0.08794872078190509
110 イタリア 0.08793929423725212
21 グアテマラ 0.08696725338758961
73 ナイジェリア 0.0864126259181962
85 トリニダード・トバゴ 0.08332883664901258
101 トーゴ 0.08314511555969396
62 南アフリカ共和国 0.08012067983037129
60 コートジボワール 0.07990180914619792
167 インドネシア 0.079437